# -------------------------                      GBM                      ----------------------------


In [2]:
import ast
import csv
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import os
import pandas as pd
import random
import seaborn as sns

%matplotlib inline
from hyperopt import fmin, hp, tpe, Trials, STATUS_OK
from hyperopt.pyll.stochastic import sample
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import KFold
from timeit import default_timer as timer

In [3]:
os.chdir(r"C:/Users/mmandziej001/Desktop/Projects/FAIT/Prediction Module/POLAND_DANE/MODELING")

In [4]:
train_data = pd.read_csv('model_training/data/data_train_under_inputed.csv')
test_data = pd.read_csv('model_training/data/data_test_under_inputed.csv')

train_id = train_data.pop('NIP')
test_id = test_data.pop('NIP')

train_labels = train_data.pop('Label')
test_labels = test_data.pop('Label')

train_datestamp = train_data.pop('DataUpadlosci')
test_datestamp = test_data.pop('DataUpadlosci')

In [18]:
#boruta_features = ['ScreenedParties', 'PartyType_Subsidiary', 'Critical_last_3_checklistsDR', 'ESR_Full_ESR_review', 'Minor_last_3_checklistsDR', 'Minor_last_3_checklistsPC', 'Major_last_3_checklistsPC', 'TLAssignedName_Yadav__N___Neha_', 'Cases_last_30_days_of_PC', 'Minor_last_10_checklistsDR', 'Critical_last_10_checklistsDR', 'Cases_last_5_days_of_PC', 'TLAssignedName_Makowska__M_M___Malgorzata_', 'TLAssignedName_Michalik__J___Justyna_', 'TeamExperience', 'HourNumeric', 'GroupCases', 'TLAssignedName_Jurojc__M___Mateusz_', 'ProcessingUnit_Gdansk', 'ProcessingUnit_MidCorp', 'FirstGroupCase', 'TLAssignedName_Jastrzebowska__S___Sonia_', 'Minor_last_10_checklistsPC', 'Major_last_10_checklistsPC', 'TLAssignedName_Marcos_Cantabrana__I___Ivan_', 'TLAssignedName_Wojciechowska__M___Magdalena_', 'ProjectExperience', 'Cases_last_30_days_of_DR', 'Cases_last_5_days_of_DR', 'TLAssignedName_Rybka__I_A___Izabela_Anna_', 'Major_last_10_checklistsDR', 'Major_last_3_checklistsDR', 'Major_last_5_checklistsDR']

In [13]:
boruta_importance = pd.read_excel('model_training/R_DUMPS/Boruta_variable_importance_under.xlsx')
boruta_importance = boruta_importance.sort_values(by=['normHits', 'meanImp'],
                                                  ascending=True)
boruta_features = list(
    boruta_importance[boruta_importance['decision'] == 'Confirmed'].V7)
boruta_pred_power = list(boruta_importance.V7)
boruta_features = [c for c in boruta_features if c not in [
    'DeclaredAccountsCount', 'RemovalDaysAgo',
    'X6', 'RevenueToCash', 'P4', 'RevenueToWages']]

In [14]:
test_data = test_data[boruta_features] # used_columns, boruta_features
train_data = train_data[boruta_features] # used_columns, boruta_features

In [15]:
print(list(train_data))
print(len(train_labels), sum(train_labels), round(sum(train_labels)/len(train_labels), 2))
print(len(test_labels), sum(test_labels), round(sum(test_labels)/len(test_labels), 2))

['EMISLegalForm_Other', 'ExpiredLicenses', 'ActiveLicenses', 'NoMail_NIE', 'FormaWlasnosci_WŁASNOŚĆ PRYWATNA KRAJOWA POZOSTAŁA', 'FormaWlasnosci_WŁASNOŚĆ ZAGRANICZNA', 'VirtualAccountsPresence_NIE', 'NoMail_TAK', 'PhoneNotPresent_NIE', 'PhoneNotPresent_TAK', 'FormaWlasnosci_Other', 'FormaWlasnosci_WŁASNOŚĆ KRAJOWYCH OSÓB FIZYCZNYCH', 'SpecialLegalForm_SPÓŁKI JAWNE', 'VirtualAccountsPresence_TAK', 'AdresBiuroWirtualne_NIE', 'RyzykownaDziałalnoscGlowna_NIE', 'AdresLokal_TAK', 'NoFax_TAK', 'NoFax_NIE', 'CAACEksport_NIE', 'NoWebsite_NIE', 'NoWebsite_TAK', 'AdresLokal_NIE', 'CAACImport_NIE', 'AffiliatesCount', 'DescriptionNull_NIE', 'AdresLokal_BrakDanych', 'DescriptionNull_TAK', 'MainPKD_BrakDanych', 'CAACImport_BrakDanych', 'CAACEksport_BrakDanych', 'RyzykowneDzialalnosciDodatkowe_BrakDanych', 'RyzykownaDziałalnoscGlowna_BrakDanych', 'AdresBiuroWirtualne_BrakDanych', 'NonCurrentLiabilities', 'ExternalIdsOthers', 'EMISLegalForm_PL-SK', 'RepresentationCount', 'RetainedEarnings', 'CashandCas

In [23]:
train_features = np.array(train_data)
test_features = np.array(test_data)

In [9]:
results_GBM = []

In [ ]:
for lr in range(60, 125, 10):
    for ne in range(13000, 22000, 1000):
        model = lgb.LGBMClassifier(max_depth=1, learning_rate=lr/100000, n_estimators=ne, is_unbalance=True)#,is_unbalance=True
        model.fit(train_data, train_labels)
        test_predictions = model.predict_proba(test_data, raw_score=True)
        train_predictions = model.predict_proba(train_data, raw_score=True)
        auc_test = roc_auc_score(test_labels, test_predictions)
        auc_train = roc_auc_score(train_labels, train_predictions)
        overfit = round(auc_train - auc_test, 4)
        results_GBM.append({'max_depth': 1,
                            'learning_rate': lr/100000,
                            'n_estimators': ne, 
                            'auc_train': auc_train,
                            'auc_test': auc_test,
                            'overfit': overfit})
        print('Learning rate:', lr/10000, 'N estimators:', ne, 'AUC train:',
              round(auc_train, 3), 'AUC test:', round(auc_test, 3), 'Overfit:', overfit)

basic model - baseline for auc

In [16]:
# save default parameters
# model = lgb.LGBMClassifier(max_depth=1, learning_rate=0.001 , n_estimators=15000)#,is_unbalance=True
# model = lgb.LGBMClassifier(max_depth=1, learning_rate=0.0006 , n_estimators=13200)#,is_unbalance=True

model = lgb.LGBMClassifier(max_depth=1, learning_rate=0.001, n_estimators=15000) #,is_unbalance=True)
model.fit(train_data, train_labels)
# train_time = timer() - start
test_predictions = model.predict_proba(test_data, raw_score=True)
train_predictions = model.predict_proba(train_data, raw_score=True)
auc_test = roc_auc_score(test_labels, test_predictions)
auc_train = roc_auc_score(train_labels, train_predictions)
overfit = round(auc_train - auc_test, 4)
print('AUC train:', round(auc_train, 3), 'AUC test:', round(auc_test, 3), 'Overfit:', overfit)

AUC train: 0.932 AUC test: 0.93 Overfit: 0.0022


In [103]:
results_df = pd.DataFrame(results_GBM)
results_df.to_excel(r'model_training\results\PC\gbm\gbm_cat_all_boruta.xlsx')

In [17]:
test_predictions_df = pd.DataFrame(test_predictions)
train_predictions_df = pd.DataFrame(train_predictions)

test_results = pd.concat([test_id, test_datestamp, test_predictions_df, test_labels], axis=1)
test_results = test_results.rename(columns={0: "Score"})
test_results = test_results.sort_values(by='Score', axis=0, ascending=False)

SumTest = pd.DataFrame(np.cumsum(test_results['Label']))
SumTest = SumTest.rename(columns={'Label': 'Sum'})
test_results = pd.concat([test_results, SumTest], axis=1)
test_results.reset_index(inplace = True , drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(test_results.shape[0]+1))/(test_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(test_results['Sum']/test_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
test_results = pd.concat([test_results, X_coordinates,y_coordinates], axis=1)

# train results
train_results = pd.concat([train_id, train_datestamp, train_predictions_df, train_labels], axis=1)
train_results = train_results.rename(columns={0:"Score"})
train_results = train_results.sort_values(by='Score', axis=0, ascending=False)

SumTrain = pd.DataFrame(np.cumsum(train_results['Label']))
SumTrain = SumTrain.rename(columns={'Label':'Sum'})
train_results = pd.concat([train_results, SumTrain], axis=1)

train_results.reset_index(inplace = True , drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(train_results.shape[0]+1))/(train_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(train_results['Sum']/train_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
train_results = pd.concat([train_results, X_coordinates,y_coordinates], axis=1)

In [18]:
test_results.to_csv(r'model_training/results/gbm/gbm_test_results.csv')
train_results.to_csv(r'model_training/results/gbm/gbm_train_results.csv')
model.booster_.save_model(r'model_training/results/gbm/gbm_best_model.txt')
#model = lgb.Booster(model_file=r'gbm_best_model.txt')
print(list(train_data.columns))
with open('model_training/results/gbm/used_features.txt', 'w') as f:
    for item in train_data.columns:
        f.write("%s\n" % item)

['EMISLegalForm_Other', 'ExpiredLicenses', 'ActiveLicenses', 'NoMail_NIE', 'FormaWlasnosci_WŁASNOŚĆ PRYWATNA KRAJOWA POZOSTAŁA', 'FormaWlasnosci_WŁASNOŚĆ ZAGRANICZNA', 'VirtualAccountsPresence_NIE', 'NoMail_TAK', 'PhoneNotPresent_NIE', 'PhoneNotPresent_TAK', 'FormaWlasnosci_Other', 'FormaWlasnosci_WŁASNOŚĆ KRAJOWYCH OSÓB FIZYCZNYCH', 'SpecialLegalForm_SPÓŁKI JAWNE', 'VirtualAccountsPresence_TAK', 'AdresBiuroWirtualne_NIE', 'RyzykownaDziałalnoscGlowna_NIE', 'AdresLokal_TAK', 'NoFax_TAK', 'NoFax_NIE', 'CAACEksport_NIE', 'NoWebsite_NIE', 'NoWebsite_TAK', 'AdresLokal_NIE', 'CAACImport_NIE', 'AffiliatesCount', 'DescriptionNull_NIE', 'AdresLokal_BrakDanych', 'DescriptionNull_TAK', 'MainPKD_BrakDanych', 'CAACImport_BrakDanych', 'CAACEksport_BrakDanych', 'RyzykowneDzialalnosciDodatkowe_BrakDanych', 'RyzykownaDziałalnoscGlowna_BrakDanych', 'AdresBiuroWirtualne_BrakDanych', 'NonCurrentLiabilities', 'ExternalIdsOthers', 'EMISLegalForm_PL-SK', 'RepresentationCount', 'RetainedEarnings', 'CashandCas

# Bayesian hyperoptimalization - has a tendency to overfit, therefore watch out

In [118]:


N_FOLDS=3


def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, lgb.Dataset(train_data,train_labels), num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

In [119]:
# Create the learning rate
learning_rate = {'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.5))}
num_leaves = {'num_leaves': hp.quniform('num_leaves', 20, 250, 1)}

In [120]:
boosting_type = {'boosting_type': hp.choice('boosting_type', 
                                            [{'boosting_type': 'gbdt', 'subsample': hp.uniform('subsample', 0.5, 1)}, 
                                             {'boosting_type': 'dart', 'subsample': hp.uniform('subsample', 0.5, 1)},
                                             {'boosting_type': 'goss', 'subsample': 1.0}])}
# Draw a sample
params = sample(boosting_type)

In [121]:
subsample = params['boosting_type'].get('subsample', 1.0)

# Extract the boosting type
params['boosting_type'] = params['boosting_type']['boosting_type']
params['subsample'] = subsample

In [122]:
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'max_depth' : hp.choice('max_depth',np.arange(1, 14, dtype=int)),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}


In [123]:
tpe_algorithm = tpe.suggest
bayes_trials = Trials()

out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

In [124]:
%%capture

# Global variable
global  ITERATION

ITERATION = 10
MAX_EVALS=50
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(50))

In [125]:
# Sort the trials with lowest loss (highest AUC) first
#bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
results = pd.read_csv('gbm_trials.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()


# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

{'boosting_type': 'goss',
 'class_weight': 'balanced',
 'colsample_bytree': 0.7292016631698011,
 'learning_rate': 0.010705097915397449,
 'min_child_samples': 20,
 'num_leaves': 39,
 'reg_alpha': 0.6689164372874457,
 'reg_lambda': 0.5471632266906857,
 'subsample_for_bin': 100000,
 'subsample': 1.0}

In [126]:
best_bayes_estimators = int(results.loc[0, 'estimators'])
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()

best_bayes_model = lgb.LGBMClassifier(n_estimators=best_bayes_estimators, n_jobs = -1, 
                                       objective = 'binary', random_state = 50, **best_bayes_params,max_depth=5)

best_bayes_model.fit(train_data, train_labels)

test_features=np.array(test_data)
train_features=np.array(train_data)
test_predictions = best_bayes_model.predict_proba(test_features)[:, 1]
train_predictions=best_bayes_model.predict_proba(train_features)[:, 1]


#auc_best_model = roc_auc_score(test_labels, test_predictions)
auc_best_model = roc_auc_score(test_labels, test_predictions)
auc_best_model

0.7069978597622248

In [37]:
best_bayes_params

{'boosting_type': 'goss',
 'class_weight': 'balanced',
 'colsample_bytree': 0.7292016631698011,
 'learning_rate': 0.010705097915397449,
 'min_child_samples': 20,
 'num_leaves': 39,
 'reg_alpha': 0.6689164372874457,
 'reg_lambda': 0.5471632266906857,
 'subsample_for_bin': 100000,
 'subsample': 1.0}

In [127]:
auc_best_model_train = roc_auc_score(train_labels, train_predictions)
auc_best_model_train

0.7967594193300186

In [29]:
test_predictions = model.predict_proba(test_features)[:, 1]
train_predictions=model.predict_proba(train_features)[:, 1]

test_predictions_df = pd.DataFrame(test_predictions)
train_predictions_df = pd.DataFrame(train_predictions)

test_results = pd.concat([test_id, test_datestamp test_predictions_df, test_labels], axis=1)
test_results = test_results.rename(columns={0: "Score"})
test_results = test_results.sort_values(by='Score', axis=0, ascending=False)

SumTest = pd.DataFrame(np.cumsum(test_results['Label']))
SumTest = SumTest.rename(columns={'Label': 'Sum'})
test_results = pd.concat([test_results, SumTest], axis=1)
test_results.reset_index(inplace = True , drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(test_results.shape[0]+1))/(test_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(test_results['Sum']/test_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
test_results = pd.concat([test_results, X_coordinates,y_coordinates], axis=1)

# train results
train_results = pd.concat([train_id, train_datestamp, train_predictions_df, train_labels], axis=1)
train_results = train_results.rename(columns={0:"Score"})
train_results = train_results.sort_values(by='Score', axis=0, ascending=False)

SumTrain = pd.DataFrame(np.cumsum(train_results['Label']))
SumTrain = SumTrain.rename(columns={'Label':'Sum'})
train_results = pd.concat([train_results, SumTrain], axis=1)

train_results.reset_index(inplace = True , drop=True)
X_coordinates = pd.DataFrame(np.arange(1,(train_results.shape[0]+1))/(train_results.shape[0]))
X_coordinates = X_coordinates.rename(columns={0: 'X_coordinates'})
y_coordinates = pd.DataFrame(train_results['Sum']/train_results['Sum'].max())
y_coordinates = y_coordinates.rename(columns={'Sum': 'y_coordinates'})
train_results = pd.concat([train_results, X_coordinates,y_coordinates], axis=1)

test_results.to_csv(r'gbm_test_results.csv')
train_results.to_csv(r'gbm_train_results.csv')

In [30]:
model.booster_.save_model(r'gbm_best_model.txt')

In [386]:
#model = lgb.Booster(model_file=r'C:/Users/kzielinski003/Desktop/Chocolate/gbm_best_model_major.txt')